# convert from tsv to csv
from __future__ import print_function, division, absolute_import

import sys
import csvkit as csv

file_name = "messer_SHC_note_final_2.tsv"    
out_file = open("SHC_note2.csv",'w')
i=0
with open(file_name,'rb') as tsvin, open('new.csv', 'wb') as csvout:
    tsvin = csv.reader(tsvin, delimiter='\t')
    csvout = csv.writer(csvout)

    for row in tsvin:
        #print(type(row))
        if len(row) > 0:
            csvout.writerow(row)
        #i += 1
        #if i > 3:
        #    break


#get rid of progress notes and others
from __future__ import print_function, division, absolute_import

import sys
import csvkit as csv

file_name = "SHC_ALL_NOTES_RAW.csv"    
out_file = open("SHC_ALL_NOTES_FILTERED.csv",'w')
i=1
        
with open(file_name) as cf: 
    reader = csv.DictReader(cf) 
    outfieldnames = reader.fieldnames 
    writer = csv.DictWriter(out_file, fieldnames=outfieldnames, restval='*') 
    writer.writeheader()
    for line in reader:
        i += 1
        eeg_no=""
        if (line['Note'] is None) or (len(line['Note']) == 0):
            print("malformed at %s", i)
            continue
        notebody = line['Note'].lower()
        notetype = line['Note_type_Desc'].lower()
        if "EEG Number" in notebody:
            writer.writerow(line)
            continue
        if "standard international system 10-20" in notebody:
            writer.writerow(line)
            continue
        if "standard international 10-20" in notebody:
            writer.writerow(line)
            continue
        if "conditions of recording" in notebody:
            writer.writerow(line)
            continue
        if "conditions of the recording" in notebody:
            writer.writerow(line)
            continue
        if "eeg #" in notebody:
            writer.writerow(line)
            continue
            
        if "neurodiagnostic" in notetype:
            writer.writerow(line)
            continue
        if "outpatient procedure" in notetype:
            writer.writerow(line)
            continue
        #print(type(line['Note']))
        #print("line %s" + notebody[:100], i )
        #print()
        #if i > 1900:
        #    break
    #print(eeg_no)    
    #i += 1
    #if (i>600):
    #    break

# normal versus abn
from __future__ import print_function, division, absolute_import

import sys
import csvkit as csv
import re
import string

file_name = "SHC_ALL_NOTES_FILTERED.csv"    
out_file = open("SHC_ALL_NOTES_IMPRESS.csv",'w')
eegImprPattern = r'(?:\
Impression and Summary:|IMPRESSION and SUMMARY:|IMPRESSION and CONCLUSIONS:\
|IMPRESSION|IMPRESSIONS|IMPRESSION:|Impression:|IMPRESSIONS:\
|INTERPRETATIONS:|INTERPRETATION:|Interpretation:\
|CONCLUSION:|CONCLUSIONS:|Conclusion:\
|Summary:|SUMMARY:)\s*(?P<eegno>[\w\d \.\-\(\),]+)'
re_eegno = re.compile(eegImprPattern, re.DOTALL|re.MULTILINE)
eegImprPattern = r'(?:\
Impression and Summary:|IMPRESSION and SUMMARY:|IMPRESSION and CONCLUSIONS:\
|IMPRESSION|IMPRESSIONS|IMPRESSION:|Impression:|IMPRESSIONS:|Impression|impression:\
|INTERPRETATIONS:|INTERPRETATION:|Interpretation\
|CONCLUSION:|CONCLUSIONS:|Conclusion:\
|Summary:|SUMMARY:|SUMMARY|Summary)\s*(?P<eegno>[\w\d \.\-\(\),]+)'
re_eegnoLoose = re.compile(eegImprPattern, re.DOTALL|re.MULTILINE)


def findTrueImpression(eeg_no,line,i):
    wordlist = eeg_no.lower().split()
    words = [''.join(c for c in s if c not in string.punctuation) for s in wordlist]
    #print(words)
    if "normal" in words and "abnormal" in words:
        #equivocal
        result = findSpecificPhrases(eeg_no,line,i)
        if (result == False):
            line['impression'] = "unknown"
            print(repr(i) + "EQUIVOCAL"+  eeg_no)
            print()
            
    elif "abnormal" in words:
        line['impression'] = "abnormal"
    elif "status" in words:
        line['impression'] = "abnormal"
    elif "seizure" in words and "nonepileptic" in words:
        result = findSpecificPhrases(eeg_no,line,i)
        if (result == False):
            line['impression'] = "unknown"
            print(repr(i)  +"UNKNOWN-"+ eeg_no)
            print()
    elif "nonepileptic" in words:
            line['impression'] = "normal"
    elif "seizure" in words:
            line['impression'] = "abnormal"
    elif "slowing" in words:
        line['impression'] = "abnormal"
    elif "normal" in words:
        line['impression'] = "normal"
    else: 
        line['impression'] = eeg_no
    #print("(" + repr(i) + ")" + line['impression'] + "-" + eeg_no.lower())

    
    
    
def findSpecificPhrases(eeg_no,line,i):
    if "This EEG is normal" in eeg_no:
        line['impression'] = "normal"
    elif "is normal for age" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG is within the normal" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG is within the broad normal" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG recording was normal" in eeg_no:
        line['impression'] = "normal"            
    elif "This record is normal" in eeg_no:
        line['impression'] = "normal"  
    elif "EEG recording is normal" in eeg_no:
        line['impression'] = "normal"            
    elif "This is a normal" in eeg_no:
        line['impression'] = "normal"            
         

    elif "This EEG is abnormal" in eeg_no:
        line['impression'] = "abnormal"           
    elif "is abnormal because of" in eeg_no:
        line['impression'] = "abnormal"     
    elif "recording is abnormal" in eeg_no:
        line['impression'] = "abnormal"      
    elif "markedly abnormal" in eeg_no.lower():
        line['impression'] = "abnormal"      
    elif "is abnormal due to" in eeg_no:
        line['impression'] = "abnormal"  
    elif "This record is abnormal" in eeg_no:
        line['impression'] = "abnormal"  
    elif "This is an abnormal" in eeg_no:
        line['impression'] = "abnormal"  
        

    else:
        return False        

        
i=1        
with open(file_name) as cf: 
    reader = csv.DictReader(cf) 
    outfieldnames = reader.fieldnames 
    outfieldnames.append('impression')     
    writer = csv.DictWriter(out_file, fieldnames=outfieldnames, restval='*') 
    writer.writeheader()
    for line in reader:
        i += 1
        eeg_no=""
        m = re_eegno.search(line['note'])
        if m:
            eeg_no = m.group('eegno')
            findTrueImpression(eeg_no,line,i)
        else:
            #try a looser find that may introduce more false information (more sensitive less specific pattern)
            m = re_eegnoLoose.search(line['note'])
            if m:
                eeg_no = m.group('eegno')
                findTrueImpression(eeg_no,line,i)
            elif "preliminary" in line['note'].lower():
                line['impression'] = "prelim"
            elif "prelim" in line['note'].lower():
                line['impression'] = "prelim"
            else:
                print("###IMPRESSION BLOCK MISSING-" + repr(i) + line['note'])
                print()

        writer.writerow(line)    
        #print(type(line['Note']))
        #print("line %s" + notebody[:100], i )
        #print()
        #if i > 4446:
        #    break
    #print(eeg_no)    
    #if (i>600):
    #    break


    


In [8]:
# normal versus abn AND remove impression block from notebody
from __future__ import print_function, division, absolute_import

import sys
import csvkit as csv
import re
import string
from pprint import pprint
import nltk
import yaml
import sys
import os
import re
import datetime

file_name = "SHC_ALL_NOTES_FILTERED.csv"    
out_file = open("SHC_ALL_NOTES_IMPRESS.csv",'w')
eegImprPattern = r'(?:\
Impression and Summary:|IMPRESSION and SUMMARY:|IMPRESSION and CONCLUSIONS:\
|IMPRESSION|IMPRESSIONS|IMPRESSION:|Impression:|IMPRESSIONS:\
|INTERPRETATIONS:|INTERPRETATION:|Interpretation:\
|CONCLUSION:|CONCLUSIONS:|Conclusion:\
|Summary:|SUMMARY:)\s*(?P<eegno>[\w\d \.\-\(\),]+)'
re_eegno = re.compile(eegImprPattern, re.DOTALL|re.MULTILINE)
eegImprPattern = r'(?:\
Impression and Summary:|IMPRESSION and SUMMARY:|IMPRESSION and CONCLUSIONS:\
|IMPRESSION|IMPRESSIONS|IMPRESSION:|Impression:|IMPRESSIONS:|Impression|impression:\
|INTERPRETATIONS:|INTERPRETATION:|Interpretation\
|CONCLUSION:|CONCLUSIONS:|Conclusion:\
|Summary:|SUMMARY:|SUMMARY|Summary)\s*(?P<eegno>[\w\d \.\-\(\),]+)'
re_eegnoLoose = re.compile(eegImprPattern, re.DOTALL|re.MULTILINE)

eegTypePattern = r'[a-z]{1}[0-9]+\-[0-9]+'
re_eegnoType = re.compile(eegTypePattern, re.DOTALL|re.IGNORECASE)

eegDurationPattern = r'Duration:\s*([\d]+:[\d]+:[\d]+)'
re_eegnoDuration = re.compile(eegDurationPattern, re.DOTALL|re.IGNORECASE)
eegDurationPattern2 = r'Duration of Study:\s*(([\d]+)\s*hour|([\d]+)\s*day)'
re_eegnoDuration2 = re.compile(eegDurationPattern2, re.DOTALL|re.IGNORECASE)
eegDurationPattern3 = r'test dates:\s*([\d\/]+)\s*-\s*([\d\/]+)' 
re_eegnoDuration3 = re.compile(eegDurationPattern3, re.DOTALL|re.IGNORECASE)
eegDurationPattern4 = r'\s*([\d\.]+)\s*(hour(s?)|day(s?))' 
re_eegnoDuration4 = re.compile(eegDurationPattern4, re.DOTALL|re.IGNORECASE)
eegDurationPattern5 = r'test dates:\s*([\d\/]+).*?-\s*([\d\/]+)' 
re_eegnoDuration5 = re.compile(eegDurationPattern5, re.DOTALL|re.IGNORECASE)

eegAbnormalityTypePattern = ['no epileptiform', 'absence of epileptiform', 'not epileptiform']
#r'(?<!no)\s+(epileptiform)'
eegAbnormalityTypePattern2 = r'(?<!absence of)\s+(epileptiform)'
#(?:(?!\bno\b).)*?\bepileptiform\b'
#re_eegnoAbnormalityType1 = re.compile(eegAbnormalityTypePattern1, re.DOTALL|re.IGNORECASE|re.MULTILINE)

def findTrueImpression(eeg_no,line,i):
    wordlist = eeg_no.lower().split()
    words = [''.join(c for c in s if c not in string.punctuation) for s in wordlist]
    #print(words)
    if "normal" in words and "abnormal" in words:
        #equivocal
        result = findSpecificPhrases(eeg_no,line,i)
        if (result == False):
            line['impression'] = "unknown"
            print(repr(i) + "EQUIVOCAL"+  eeg_no)
            print()
            
    elif "abnormal" in words:
        line['impression'] = "abnormal"
    elif "status" in words:
        line['impression'] = "abnormal"
    elif "seizure" in words and "nonepileptic" in words:
        result = findSpecificPhrases(eeg_no,line,i)
        if (result == False):
            line['impression'] = "unknown"
            print(repr(i)  +"UNKNOWN-"+ eeg_no)
            print()
    elif "nonepileptic" in words:
            line['impression'] = "normal"
    elif "seizure" in words:
            line['impression'] = "abnormal"
    elif "slowing" in words:
        line['impression'] = "abnormal"
    elif "normal" in words:
        line['impression'] = "normal"
    else: 
        line['impression'] = eeg_no
    #print("(" + repr(i) + ")" + line['impression'] + "-" + eeg_no.lower())

    
    
    
def findSpecificPhrases(eeg_no,line,i):
    if "This EEG is normal" in eeg_no:
        line['impression'] = "normal"
    elif "is normal for age" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG is within the normal" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG is within the broad normal" in eeg_no:
        line['impression'] = "normal"            
    elif "EEG recording was normal" in eeg_no:
        line['impression'] = "normal"            
    elif "This record is normal" in eeg_no:
        line['impression'] = "normal"  
    elif "EEG recording is normal" in eeg_no:
        line['impression'] = "normal"            
    elif "This is a normal" in eeg_no:
        line['impression'] = "normal"            
         

    elif "This EEG is abnormal" in eeg_no:
        line['impression'] = "abnormal"           
    elif "is abnormal because of" in eeg_no:
        line['impression'] = "abnormal"     
    elif "recording is abnormal" in eeg_no:
        line['impression'] = "abnormal"      
    elif "markedly abnormal" in eeg_no.lower():
        line['impression'] = "abnormal"      
    elif "is abnormal due to" in eeg_no:
        line['impression'] = "abnormal"  
    elif "This record is abnormal" in eeg_no:
        line['impression'] = "abnormal"  
    elif "This is an abnormal" in eeg_no:
        line['impression'] = "abnormal"  
        

    else:
        return False        


def setImpressionBody(m,line,i):
    impstr = line['note'][m.start():]
    n = re.search(r'(?:comments|comment|clinical correlation):', impstr, re.IGNORECASE)
    if n:
        line['impressionBody'] = impstr[:n.start()]
    else:
        line['impressionBody'] = impstr

    # get rid of double spaces
    line['impressionBody'] = " ".join(line['impressionBody'].split())

def setImpressionType(line,i):
    line['impressionType'] = ""        
    impstr = line['impressionBody']
    if impstr is None:
        return
    #https://regex101.com/

    n = re.search(r'(?:(?<!absence of)(?<!no)(?<!not))\s+(epileptiform)', impstr, re.IGNORECASE)
    if n:
        if ("other epileptiform" not in impstr.lower()):
            line['impressionType'] += ' ed' #epileptiform discharge

    n = re.search(r'(?:(?<!absence of)(?<!no)(?<!not))\s+(discharge|spike)', impstr, re.IGNORECASE)
    if n:
        line['impressionType'] += ' dc' #discharges which are more questionable
        
    n = re.search(r'seizure', impstr, re.IGNORECASE)
    if n:
        if ("no evidence for electrographic seizures" not in impstr.lower()) and \
           ("for a seizure disorder" not in impstr.lower()) and \
           ("no evidence for electrographic seizure" not in impstr.lower()) and \
           ("does not exclude a clinical diagnosis of seizure" not in impstr.lower()) and \
           ("no electrographic seizure" not in impstr.lower()):
            line['impressionType'] += ' sz'

def durationByDates(t1, t2):
    dtformat = '%m/%d/%Y'
    try:
        t1 = datetime.datetime.strptime(t1, dtformat)
        t2 = datetime.datetime.strptime(t2, dtformat)
        t1 = t1.date()
        t2 = t2.date()
        td = t2-t1
        return (repr(td.days*24) +":"+ repr(td.seconds//3600).zfill(2)  +":"+ repr((td.seconds//60)%60).zfill(2) )
    except:
        return ""
#print(datetime.timedelta(0, 8, 562000))
#divmod(b.days * 86400 + b.seconds, 60)
#datetime.timedelta(0, 8, 562000)

def durationByHoursHopefully(eegReport): #last ditch effort to match any mention of days or hours in the report
    hours = 0 
    days = 0
    i=0
    for match in re.finditer(eegDurationPattern4, eegReport):
        matchtxt = match.group(2)
        i += 1
        #print ("hi" + repr(i) + match.group(1) + matchtxt)
        if "hour" in matchtxt:
            hours =  int(float(match.group(1)))
        elif "day" in matchtxt:
            days = int(float(match.group(1)))
            #print("################################")
    totalhours=  hours + 24*days
    #print (totalhours)
    return repr(totalhours) + ':00:00'

In [9]:


class Splitter(object):

    def __init__(self):
        self.nltk_splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self, text):
        """
        input format: a paragraph of text
        output format: a list of lists of words.
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        """
        sentences = self.nltk_splitter.tokenize(text)
        tokenized_sentences = [self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences


class POSTagger(object):

    def __init__(self):
        pass
        
    def pos_tag(self, sentences):
        """
        input format: list of lists of words
            e.g.: [['this', 'is', 'a', 'sentence'], ['this', 'is', 'another', 'one']]
        output format: list of lists of tagged tokens. Each tagged tokens has a
        form, a lemma, and a list of tags
            e.g: [[('this', 'this', ['DT']), ('is', 'be', ['VB']), ('a', 'a', ['DT']), ('sentence', 'sentence', ['NN'])],
                    [('this', 'this', ['DT']), ('is', 'be', ['VB']), ('another', 'another', ['DT']), ('one', 'one', ['CARD'])]]
        """

        pos = [nltk.pos_tag(sentence) for sentence in sentences]
        #adapt format
        pos = [[(word, word, [postag]) for (word, postag) in sentence] for sentence in pos]
        return pos

class DictionaryTagger(object):

    def __init__(self, dictionary_paths):
        files = [open(path, 'r') for path in dictionary_paths]
        dictionaries = [yaml.load(dict_file) for dict_file in files]
        map(lambda x: x.close(), files)
        self.dictionary = {}
        self.max_key_size = 0
        for curr_dict in dictionaries:
            for key in curr_dict:
                if key in self.dictionary:
                    self.dictionary[key].extend(curr_dict[key])
                else:
                    self.dictionary[key] = curr_dict[key]
                    self.max_key_size = max(self.max_key_size, len(key))

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while (i < N):
            j = min(i + self.max_key_size, N) #avoid overflow
            tagged = False
            while (j > i):
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    #self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token: #if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence

def value_of(sentiment):
    if sentiment == 'positive': return 1
    if sentiment == 'negative': return -1
    return 0

def sentence_score_deprecated(sentence_tokens, previous_token, acum_score):    
    if not sentence_tokens:
        return acum_score
    else:
        current_token = sentence_tokens[0]
        tags = current_token[2]
        token_score = sum([value_of(tag) for tag in tags])
        if previous_token is not None:
            previous_tags = previous_token[2]
            if 'inc' in previous_tags:
                token_score *= 2.0
            elif 'dec' in previous_tags:
                token_score /= 2.0
            elif 'inv' in previous_tags:
                token_score *= -1.0
        return sentence_score(sentence_tokens[1:], current_token, acum_score + token_score)

def sentence_score(sentences):    
    scored_sentences = []
    if not sentences:
        return None
    else:
        for sentence in sentences:
            tagged_expression = []
            token_score = 0
            for (word, wordstem, postag) in sentence:
                if 'inc' in postag:
                    token_score *= 2.0
                elif 'dec' in postag:
                    token_score /= 2.0
                elif 'inv' in postag:
                    token_score += -1             
                else:
                    token_score = 0
                tagged = (word, wordstem, postag, token_score)
                tagged_expression.append(tagged)
            scored_sentences.append(tagged_expression)
    return scored_sentences

def sentence_simplify(sentences):    
    scored_sentences = []
    if not sentences:
        return None
    else:
        for sentence in sentences:
            tagged_expression = []
            for (word, wordstem, postag) in sentence:
                #print(postag)
                if ('DT' in postag) or \
                    ('IN' in postag) or \
                    ('NN' in postag) or \
                    ('RB' in postag) or \
                    ('JJ' in postag) or \
                    ('NNS' in postag):
                    tagged = (word, wordstem, postag)
                    tagged_expression.append(tagged)
                    #print('ok')
            if tagged_expression:
                scored_sentences.append(tagged_expression)
    return scored_sentences


def sentiment_score(review):
    finalized_score = 0
    if (review is None):
        return 0
    for sentence in review:
        if sentence:
            #pprint(sentence)
            token_score = 0
            for (word, wordstem, tags, score) in sentence:
                token_score += sum([value_of(tag) for tag in tags])
            #print(token_score)
            for (word, wordstem, tags, score) in sentence:
                if score != 0:
                    token_score *= score 
                    #print(token_score)
            finalized_score += token_score
    return finalized_score

        
def sentimentAnalysis(text):

    splitter = Splitter()
    postagger = POSTagger()
    dicttagger = DictionaryTagger([ 'dicts/positive.yml', 'dicts/negative.yml', 
                                    'dicts/inc.yml', 'dicts/dec.yml', 'dicts/inv.yml'])

    if isinstance(text, basestring):
        splitted_sentences = splitter.split(text)
    else:
        return 0
    #pprint(text)
    #pprint(splitted_sentences)

    pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
    #pprint(pos_tagged_sentences)

    dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
    #pprint(dict_tagged_sentences)

    dict_tagged_sentences = sentence_simplify(dict_tagged_sentences)
    #print(dict_tagged_sentences)
    #() + 1
    score_tagged_sentences = sentence_score(dict_tagged_sentences)
    #print(score_tagged_sentences)
    #print("analyzing sentiment...")
    score = sentiment_score(score_tagged_sentences)
    #print(score)    
    return score

#sentimentAnalysis("left temporal partial-onset ictal activity abnormal")

    

In [10]:
import pandas as pd
import numpy as np
from IPython.display import display
import datetime
import codecs
import csv

def loadMasterCSVFile():
    with codecs.open("NKdatabaseDump.csv", 'r', encoding='utf-8', errors='ignore') as cf:
        i = 0
        masterCSV = csv.reader(cf)
        masterCSV_list = [row for row in masterCSV]
        print(len(masterCSV_list))
        return masterCSV_list


def durationByDatesForCSV(t1, t2):
    try:
        dtformat = '%m/%d/%Y %I:%M:%S %p'
    #try:
    #print(t1)
        t1 = datetime.datetime.strptime(t1, dtformat)
        t2 = datetime.datetime.strptime(t2, dtformat)
    #print(t2)
    #t1 = t1.date()
    #t2 = t2.date()
    except ValueError:
        try:
            dtformat = '%m/%d/%Y'
            t1 = datetime.datetime.strptime(t1, dtformat)
            t2 = datetime.datetime.strptime(t2, dtformat)        
        except:
            return None
    except:
        return None
    td = t2-t1
    return td
    #print(td)
    return (repr(td.days*24) +":"+ repr(td.seconds//3600).zfill(2)  +":"+ repr((td.seconds//60)%60).zfill(2) )
    #except:
    #    return ""
    
# return a dateDelta for any given exam number by looking up all the rows with an exam number, 
# then looking for the minimal and maximal dates within those rows
def masterCSV_daterange(examNO):
    #print(len(masterCSV_list))
    #columns=masterCSV_list[0]
    #print(columns)
    df = pd.DataFrame(masterCSV_list[1:],
                    columns=masterCSV_list[0])
    #print(df)
    #HTML(df.to_html())
    #display(df)
    #df = df.drop_duplicates()
    po = df['INF_ScheduleExam@ExamNo_STR'].str.contains(examNO)
    #display(po)
    #idx = df[po].index.tolist()
    #r = df.loc(idx)
    #display(r)
    resultingDF = df.iloc[np.flatnonzero(po)]
    #display(r)
    dateList = list()
    dateList.extend(resultingDF['INF_ScheduleExam@StartTime_DT'].tolist())
    dateList.extend(resultingDF['INF_ScheduleExam@EndTime_DT'].tolist())
    #print(examNO)
    #display(dateList)
    if (len(dateList) is 0):
        return None
    max_value = max(dateList)
    min_value = min(dateList)
    #print(max_value)
    #print(min_value)
    #duration = max_value - min_value
    duration = durationByDatesForCSV(min_value, max_value)
    return duration
    print(duration)
    #mask = np.column_stack([df['INF_ScheduleExam@ExamNo_STR'].str.contains(examNO)])
    #display(mask)
    #examNo_col = [item[''] for item in masterCSV_list]
    #a = np.array(masterCSV_list)
    #i,j = np.where(a==examNO)
    #x = enumerate(masterCSV_list) 
    #print(x.value)
    #print('\n'.join('{}: {}'.format(*k) for k in enumerate(masterCSV_list)))
    #result = [i for i,x in enumerate(masterCSV_list) if x.==examNO] # => [1, 3]
    #print(i,j)

masterCSV_list = loadMasterCSVFile()
masterCSV_daterange("A17-293")

55102


datetime.timedelta(3, 2177)

In [11]:

        
i=1              
with open(file_name) as cf: 
    reader = csv.DictReader(cf) 
    #optional offset 
    #for line in reader:
    #    i += 1
    #    if (i>9425):
    #        i=1
    #        break

    outfieldnames = reader.fieldnames 
    outfieldnames.append('notebody')     
    outfieldnames.append('impressionBody')     
    outfieldnames.append('impression')     
    outfieldnames.append('notetype')     
    outfieldnames.append('duration')     
    outfieldnames.append('impressionType')     
    outfieldnames.append('sentimentScore')     
    outfieldnames.append('isSeizure')     
    writer = csv.DictWriter(out_file, fieldnames=outfieldnames, restval='*') 
    writer.writeheader()
    for line in reader:
        i += 1
        eeg_no=""
        m = re_eegno.search(line['note'])
        #write the notebody and impression
        if m:
            #print(m.start())
            #print(line['note'][:m.start()])
            #break
            line['notebody'] = line['note'][:m.start()]
            eeg_no = m.group('eegno')
            findTrueImpression(eeg_no,line,i)
            setImpressionBody(m,line,i)
        else:
            #try a looser find that may introduce more false information (more sensitive less specific pattern)
            m = re_eegnoLoose.search(line['note'])
            if m:
                line['notebody'] = line['note'][:m.start()]
                setImpressionBody(m,line,i)
                eeg_no = m.group('eegno')
                findTrueImpression(eeg_no,line,i)
            elif "preliminary" in line['note'].lower():
                line['impression'] = "prelim"
            elif "prelim" in line['note'].lower():
                line['impression'] = "prelim"
            else:
                print("###IMPRESSION BLOCK MISSING-" + repr(i) + line['note'])
                print()

                  
        # figure out the duration by cross-reference the duration of the study based on the examno
        m = re_eegnoType.search(line['note'])
        line['duration'] = None
        if m:
            #print("got csv table lookup")
            line['duration'] = masterCSV_daterange(m.group(0))
            #print(line['duration'])
        if line['duration'] is None:                
            #write the duration by some other messier means
            m = re_eegnoDuration.search(line['note'])
            if m:            
                match = m.group(1)
                line['duration'] = match
            else:
                m = re_eegnoDuration2.search(line['note'])
                if m:
                    match = m.group(1)
                    #print(match)
                    if "hour" in match:
                        line['duration'] =  m.group(2) + ':00:00'
                    elif "day" in match:
                        line['duration'] = repr(int(m.group(3))*24) + ':00:00'  
                else:
                    m = re_eegnoDuration3.search(line['note'])
                    if m:
                        line['duration'] = durationByDates(m.group(1), m.group(2))
                    else:
                        m = re_eegnoDuration4.search(line['note'])
                        if m:
                            line['duration'] = durationByHoursHopefully(line['note'])
                        else:
                            m = re_eegnoDuration5.search(line['note'])
                            print("here i am" + repr(m))
                            if m:
                                print(m.group(1))
                                line['duration'] = durationByDates(m.group(1), m.group(2))
                            
        #print(line['duration'])
        #() + 1                                
                    
                            

        #write the notetype
        m = re_eegnoType.search(line['note'])
        if m:
            match = m.group(0)
            match = match.lower()
            if ("s" in match):
                line['notetype'] = "spot"
            elif ("v" in match):
                line['notetype'] = "ceeg"
            elif ("a" in match):
                line['notetype'] = "ambu"
            elif ("f" in match):
                line['notetype'] = "spot inpt"
            elif ("e" in match):
                line['notetype'] = "ceeg"
            else:
                line['notetype'] = "unk"
                
        elif line['duration'] is not None:
            #determine the unknown report based off the duration only
            dtformat = '%H:%M:%S'
            try:
                t = datetime.datetime.strptime(line['duration'], dtformat)
                t = t.minute + t.hour*60
                if t < 100 and t > 18:
                    line['notetype'] = "spot?"
            except:
                pass

        #write the abnormality type
        setImpressionType(line,i)
            
        #write sentiment score
        line['sentimentScore'] = sentimentAnalysis(line['impressionBody'])

        if ('sz' in line['impressionType']) and int(line['sentimentScore']) < 0:
            line['isSeizure'] = 1
        elif ('sz' in line['impressionType']) ^ int(line['sentimentScore']) < 0:
            line['isSeizure'] = '?'
 
            
        #print(repr(i) + ": "+ line['impressionType'])
        
        if (line['notetype'] == "ambu"):        
            writer.writerow(line)    
        #writer.writerow(line)    
        #print(type(line['Note']))
        #print("line %s" + line['note_Id'][:20] + repr(line['sentimentScore']), i )
        #print("line %s" + line['notebody'][:200], i )
        #print()
        #if i > 555:
        #    break
    #print(eeg_no)    
    #if (i>600):
    #    break
#close(file_name)
#writer.close()

here i amNone


NameError: name 'basestring' is not defined

# -*- coding: utf-8 -*- 
from __future__ import print_function, division, absolute_import

import sys
import re

import csvkit as csv

allReports = ""
with open("SHC_ALL_NOTES_FILTERED.csv") as cf:
    reader = csv.DictReader(cf)
    for line in reader:
        allReports += line['note'] + " \n\n"
        # print(desc)
        # now manipulate the note body
        
f = open("allReports.txt", "w")
f.write(allReports)
f.close()

In [ ]:
#make a histogram of the most commonly used words in all EEG reports
from collections import Counter
from nltk.corpus import stopwords
import pandas
import matplotlib.pyplot as plt

f = open("allReports.txt", "r")
allReports = f.read() 
f.close()

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

allwords_stemmed = tokenize_and_stem(allReports)


In [ ]:
#get rid of stop words
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
filtered_words = [word for word in allwords_stemmed if word not in stopwords]

letter_counts = Counter(filtered_words)
df = pandas.DataFrame.from_dict(letter_counts, orient='index')
df = df.sort_values(by=0, ascending=0)
df1 = df[:20]
plt.show(block=True)
df1.plot(kind='bar')

In [ ]:
# most common bigrams
import nltk
from nltk import bigrams
report = allReports.lower()
reportNLTK = nltk.Text(nltk.tokenize.word_tokenize(report))
reportNLTK.collocations()

In [ ]:
# generate some eeg reports for me by sequencing bigrams
def generate_model(cfdist, word, num=15):
    for i in range(num):
        print(word, end=' ')
        word = cfdist[word].max()

import nltk
from nltk.corpus import stopwords

#text = nltk.corpus.genesis.words('english-kjv.txt')
#print(type(text))
corpus = nltk.corpus.reader.plaintext.PlaintextCorpusReader(".", "allReports.txt")
words=corpus.words()
#get rid of stop words
#filtered_words = [word for word in words if word not in stopwords.words('english')]
#words = filtered_words

#print(type(words))
bigrams = nltk.bigrams(words)
cfd = nltk.ConditionalFreqDist(bigrams)
generate_model(cfd, 'seizure')
#filtered_words = [word for word in word_list if word not in stopwords.words('english')]

In [ ]:
generate_model(cfd, 'epileptic')
print("")
generate_model(cfd, 'slow')
print("")
generate_model(cfd, 'discharge')
print("")
generate_model(cfd, 'periodic')
print("")
generate_model(cfd, 'impression')
print("")
generate_model(cfd, 'background')
print("")
generate_model(cfd, 'infantile')
print("")
generate_model(cfd, 'neonatal')
print("")
generate_model(cfd, 'seizure')
print("")
generate_model(cfd, 'seizures')
print("")
generate_model(cfd, 'This')
print("")
generate_model(cfd, 'electrographic')
print("")
generate_model(cfd, 'indicates')
print("")
generate_model(cfd, 'represent')
print("")
generate_model(cfd, 'generalized')
print("")
generate_model(cfd, 'slowing')
print("")
generate_model(cfd, 'focal')
print("")
generate_model(cfd, '3')
print("")
generate_model(cfd, 'hz')
print("")
generate_model(cfd, 'second')
print("")
generate_model(cfd, 'button')
print("")
generate_model(cfd, 'cry')
print("")
generate_model(cfd, 'clonic')
print("")
generate_model(cfd, 'tonic')
print("")
generate_model(cfd, 'partial')
print("")
generate_model(cfd, 'sleep')
print("")
generate_model(cfd, 'vertex')
print("")
generate_model(cfd, 'temporal')
print("")
generate_model(cfd, 'frontal')
print("")
generate_model(cfd, 'occipital')
print("")
#generate_model(cfd, 'hypsarrythmia')
#print("")
generate_model(cfd, 'alpha')
print("")
generate_model(cfd, 'beta')
print("")
generate_model(cfd, 'delta')
print("")
generate_model(cfd, 'theta')
print("")
generate_model(cfd, 'sudden')
print("")
generate_model(cfd, 'waxing')
print("")
generate_model(cfd, 'waning')

In [ ]:
#fraction of words in text that are not in the stopword list
def content_fraction(text):
     stopwords = nltk.corpus.stopwords.words('english')
     content = [w for w in text if w.lower() not in stopwords]
     return len(content) / len(text)

content_fraction(words)

#tutorial example of classification
import random
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000] 

def document_features(document): 
    document_words = set(document) 
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 

#tutorial continued
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 
classifier.show_most_informative_features(5)

#tutorial
#what does it containt 
movie_reviews.categories()
movie_reviews.fileids('neg')
movie_reviews.categories()
i = 1
for doc in documents:
    print(str(i) + ": "+ doc[1])
    i += 1

from nltk.corpus import names
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
print(featuresets[1])


exclude = set(string.punctuation)
print(exclude)

In [ ]:
#find the most common word associated with the interpretation result (abnormal versus normal)
#should take 10 mins
import random
import string
documents = list()
notebody = list()
result = list()
exclude = set(string.punctuation)
stopwords = nltk.corpus.stopwords.words('english')

with open("SHC_ALL_NOTES_IMPRESS.csv") as cf:
    reader = csv.DictReader(cf)
    for line in reader:
        notebody = line['note'].lower()
        notebodyStemmed = tokenize_and_stem(notebody)        
        filtered_words = [word for word in notebodyStemmed if word not in stopwords]

        #notebody = ''.join(ch for ch in notebody if ch not in exclude)
        #notebody = notebody.split() 
        #notebody = [word for word in notebody if word not in stopwords.words('english')]
        category = line['impression']
        if (category == 'abnormal' or category == 'normal'):
            documents.append((list(filtered_words), category))

#documents[1]
        # print(desc)
        # now manipulate the note body

random.shuffle(documents)
#documents[1]


i = 1
for doc in documents:
    print(doc[0])
    i += 1
    break

In [ ]:
# constructing a list of the 2000 most frequent words in the overall corpus
import nltk
words = list()
for doc in documents:
    words += doc[0] 
    
#print(words)    
all_words = nltk.FreqDist(w for w in words)
word_features = list(all_words)[:4000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

for (d,c) in documents:
    print(c)
    

In [ ]:
ccount = int (len(featuresets) / 2) - 1
#take half of the total as the training set and the other half as a test set
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[ccount:], featuresets[:ccount]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 
classifier.show_most_informative_features(30)

In [ ]:
for (d) in documents:
    print(d)
    break

# in order to load a corpus, reports must be in separate files // don't need to run this again

capture_eegno = r'(?:DATE OF SERVICE:|STUDY DATE:|DATE EEG:|Date:|Service Date:|Study date|Study dates|DATE OF EEG:|start|T:|test dates:|Date of study:|Exam date|exam date:)\s*(?P<eegno>[\d/-]+)\s*'
capture_eegno1 = r'(?:Date:|Study date |Study date:)\s*(?P<eegno>(?:Jan|Feb|Mar|Apr|May|Jun|July|Aug|Sep|Oct|Nov|Dec)\s*[,\s\d/-]+)(?:[a-z]+)'
capture_eegno2 = r'(?<!DOB:  )(?P<eegno>[\d]+/[\d]+/[\d]+)' #DOB:  10/04/1993   
eegDateRange = r'(?P<eegno>[\d/]+-[\d/]+)'
eegDateStrict = r'(?P<eegno>[\d]+/[\d]+/[\d]+)'
#capture_eegno = r'(?P<eegno>[0-9]+)\s*'
re_eegno = re.compile(capture_eegno, re.DOTALL|re.MULTILINE|re.IGNORECASE)
re_eegno1 = re.compile(capture_eegno1, re.DOTALL|re.MULTILINE|re.IGNORECASE)
re_eegno2 = re.compile(capture_eegno2, re.DOTALL|re.MULTILINE|re.IGNORECASE)
eegDateRange = re.compile(eegDateRange, re.DOTALL|re.MULTILINE|re.IGNORECASE)
eegDateStrict = re.compile(eegDateStrict, re.DOTALL|re.MULTILINE|re.IGNORECASE)

out_file = open("lpch_eeg_reports_interp_date_impression.csv",'w')

i=2
with open("lpch_eeg_reports_interp_impression.csv") as cf:
    reader = csv.DictReader(cf)
    outfieldnames = reader.fieldnames
    outfieldnames.append('date')
    writer = csv.DictWriter(out_file, fieldnames=outfieldnames, restval='***')
    writer.writeheader()
    
    for line in reader:
        eeg_no=""
        m = re_eegno.search(line['note'])
        print(str(i) + ": ", end='')
        #print(str(i) + ": " + str(type(m)), end='')
        if m:
            eeg_no = m.group('eegno')
            #print(eeg_no)
        else:
            m = re_eegno1.search(line['note'])
            if m:
                eeg_no = m.group('eegno')
                #print(eeg_no)
            else:
                #print("!!!!!!!!!!!")
                for match in re.finditer(capture_eegno2,line['note']):
                    eeg_no = match.group('eegno')
                    
                #m = re_eegno2.search(line['note'])
                #if m:
                #    eeg_no = m.group('eegno')
                #print(eeg_no +"???????")

        #if len(eeg_no) > 12:
        #    m = eegDateRange.search(eeg_no)            
        #    if m:
        #        print("bad")
        #        m = eegDateStrict.search(eeg_no)
        #        m = eegDateStrict.search(eeg_no)
        #        if m:
        #            eeg_no = m.group('eegno')
        if ((len(eeg_no) < 5) | (len(eeg_no) > 12)):
            #print("possibly bad: " + eeg_no)
            for match in re.finditer(capture_eegno2,line['note']):
                eeg_no = match.group('eegno')
        line['date'] = eeg_no 
        writer.writerow(line)
        
        #print(eeg_no)    
        #i += 1
        #if (i>600):
        #    break

In [47]:
capture_eegno = r'(?:EEG TYPE:)\s*(?P<eegno>.+?)(?:history|report|clinical|condition|patient|location)'
capture_eegno1 = r'(?:Date:|Study date |Study date:)\s*(?P<eegno>(?:Jan|Feb|Mar|Apr|May|Jun|July|Aug|Sep|Oct|Nov|Dec)\s*[,\s\d/-]+)(?:[a-z]+)'
capture_eegno2 = r'(?<!DOB:  )(?P<eegno>[\d]+/[\d]+/[\d]+)' #DOB:  10/04/1993   
eegDateRange = r'(?P<eegno>[\d/]+-[\d/]+)'
eegDateStrict = r'(?P<eegno>[\d]+/[\d]+/[\d]+)'
#capture_eegno = r'(?P<eegno>[0-9]+)\s*'
re_eegno = re.compile(capture_eegno, re.DOTALL|re.IGNORECASE)
re_eegno1 = re.compile(capture_eegno1, re.DOTALL|re.MULTILINE|re.IGNORECASE)
re_eegno2 = re.compile(capture_eegno2, re.DOTALL|re.MULTILINE|re.IGNORECASE)
eegDateRange = re.compile(eegDateRange, re.DOTALL|re.MULTILINE|re.IGNORECASE)
eegDateStrict = re.compile(eegDateStrict, re.DOTALL|re.MULTILINE|re.IGNORECASE)

out_file = open("lpch_eeg_reports_interp_date_impression_type.csv",'w')

def detectType(eeg_no):
    if re.search("ambulatory", eeg_no, re.IGNORECASE):
        return "amb"
    if re.search("routine", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("routine", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("portable", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("spot", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("out", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("sleep", eeg_no, re.IGNORECASE):
        return "spot"
    if re.search("continuous", eeg_no, re.IGNORECASE):
        return "ltm"
    if re.search("video", eeg_no, re.IGNORECASE):
        return "ltm"
    if re.search("intraoperative", eeg_no, re.IGNORECASE):
        return "iom"
    if re.search("electrocort", eeg_no, re.IGNORECASE):
        return "iom"
    if re.search("amplitude", eeg_no, re.IGNORECASE):
        return "aeeg"
    if re.search("aeeg", eeg_no, re.IGNORECASE):
        return "aeeg"
    return ""

i=2
with open("lpch_eeg_reports_interp_date_impression.csv") as cf:
    reader = csv.DictReader(cf)
    outfieldnames = reader.fieldnames
    outfieldnames.append('type')
    writer = csv.DictWriter(out_file, fieldnames=outfieldnames, restval='***')
    writer.writeheader()
    
    for line in reader:
        eeg_no=""
        m = re_eegno.search(line['note'])
        print(str(i) + ": ", end='')
        i += 1
        #print(str(i) + ": " + str(type(m)), end='')
        if m:
            eeg_no = m.group('eegno')
            eeg_no = detectType(eeg_no)
            
        if (len(eeg_no) == 0):
            eeg_no = detectType(line['note'])

        line['type'] = eeg_no
        
        writer.writerow(line)
        #if (i>1556): 
        #    break

2: 3: 4: 5: 6: 7: 8: 9: 10: 11: 12: 13: 14: 15: 16: 17: 18: 19: 20: 21: 22: 23: 24: 25: 26: 27: 28: 29: 30: 31: 32: 33: 34: 35: 36: 37: 38: 39: 40: 41: 42: 43: 44: 45: 46: 47: 48: 49: 50: 51: 52: 53: 54: 55: 56: 57: 58: 59: 60: 61: 62: 63: 64: 65: 66: 67: 68: 69: 70: 71: 72: 73: 74: 75: 76: 77: 78: 79: 80: 81: 82: 83: 84: 85: 86: 87: 88: 89: 90: 91: 92: 93: 94: 95: 96: 97: 98: 99: 100: 101: 102: 103: 104: 105: 106: 107: 108: 109: 110: 111: 112: 113: 114: 115: 116: 117: 118: 119: 120: 121: 122: 123: 124: 125: 126: 127: 128: 129: 130: 131: 132: 133: 134: 135: 136: 137: 138: 139: 140: 141: 142: 143: 144: 145: 146: 147: 148: 149: 150: 151: 152: 153: 154: 155: 156: 157: 158: 159: 160: 161: 162: 163: 164: 165: 166: 167: 168: 169: 170: 171: 172: 173: 174: 175: 176: 177: 178: 179: 180: 181: 182: 183: 184: 185: 186: 187: 188: 189: 190: 191: 192: 193: 194: 195: 196: 197: 198: 199: 200: 201: 202: 203: 204: 205: 206: 207: 208: 209: 210: 211: 212: 213: 214: 215: 216: 217: 218: 219: 220: 221: 222: 2

In [48]:
# in order to load a corpus, reports must be in separate files
from dateutil import parser

i=2
with open("lpch_eeg_reports_interp_date_impression.csv") as cf:
    reader = csv.DictReader(cf)
    
    for line in reader:
        date_str= line['date']
        try:
            dateObj = parser.parse(date_str)
            #print(dateObj.date())
            dateFile = (str(dateObj.date()))
            f = open("reports/" + dateFile + ".txt", "a+")
            f.write(line['note'])
            f.close()
        except:
            print("Unexpected error:", str(sys.exc_info()[0]) + date_str)

Unexpected error: <type 'exceptions.ValueError'>
Unexpected error: <type 'exceptions.ValueError'>0/01/1975
Unexpected error: <type 'exceptions.ValueError'>4/6-7/2012
Unexpected error: <type 'exceptions.ValueError'>
Unexpected error: <type 'exceptions.ValueError'>11/07-8/2013
Unexpected error: <type 'exceptions.ValueError'>316501658
Unexpected error: <type 'exceptions.ValueError'>318488965
Unexpected error: <type 'exceptions.ValueError'>6/30//2014
Unexpected error: <type 'exceptions.ValueError'>7/24-25/2015
Unexpected error: <type 'exceptions.ValueError'>8/6-8/7/2015
Unexpected error: <type 'exceptions.ValueError'>8/12-8/14/15
Unexpected error: <type 'exceptions.ValueError'>9/1-9/3/2015
Unexpected error: <type 'exceptions.ValueError'>9/1-9/2/2015
Unexpected error: <type 'exceptions.ValueError'>9/2-9/3/2015
Unexpected error: <type 'exceptions.ValueError'>9/2-9/3/2015
Unexpected error: <type 'exceptions.ValueError'>8/31-9/2/15
Unexpected error: <type 'exceptions.ValueError'>9/8-9/9/2015
U

In [ ]:
import nltk
import os
from nltk.corpus import PlaintextCorpusReader

#fileids = os.listdir("reports")
corpus_root = 'reports'
wordlists = PlaintextCorpusReader(corpus_root, '.*')
wordlists.fileids()

In [ ]:
cfd = nltk.ConditionalFreqDist(
           (target, fileid[:10])
           for fileid in wordlists.fileids()
           for w in wordlists.words(fileid)
           for target in ['grda', 'gpd']
           if w.lower().startswith(target))
cfd.plot()

In [ ]:
cfd.tabulate(conditions=['grda','gpd'], samples=range(10), cumulative=True)

In [ ]:
#see if there are natural clusters, unsupervised
#ngram of word, each one is dimension of word, then 
#applied text analysis python machine learning approaches on slack

In [ ]:
# try tagging 

textAll_tokenized = list()
with open("lpch_eeg_reports_interp_impression.csv") as cf:
    reader = csv.DictReader(cf)
    for line in reader:
        #print(line['note'])
        text_raw = line['note'].lower()
        text_tokenized = nltk.tokenize.word_tokenize(text_raw)
        textAll_tokenized += text_tokenized
        #words = list(text_raw.split())
        #print(words)
        #break

text = nltk.Text(textAll_tokenized)
text_tagged = nltk.pos_tag(text)

        #print(text[10:200])
print(text.similar('alpha'))



which of these tags are the most common

Tag	Meaning	English Examples
ADJ	adjective	new, good, high, special, big, local
ADP	adposition	on, of, at, with, by, into, under
ADV	adverb	really, already, still, early, now
CONJ	conjunction	and, or, but, if, while, although
DET	determiner, article	the, a, some, most, every, no, which
NOUN	noun	year, home, costs, time, Africa
NUM	numeral	twenty-four, fourth, 1991, 14:24
PRT	particle	at, on, out, over per, that, up, with
PRON	pronoun	he, their, her, its, my, I, us
VERB	verb	is, say, told, given, playing, would

In [ ]:
nltk.help.upenn_tagset('NN.*')
nltk.help.upenn_tagset('JJ.*')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('JJ')


In [ ]:
tag_fd = nltk.FreqDist(tag for (word, tag) in text_tagged)
tag_fd.most_common()

In [ ]:
word_tag_pairs = nltk.bigrams(text_tagged)
#i = 0
#for (a, b) in word_tag_pairs:
#    print(b[1])
#    i += 1
#    if i>22:
#        break
noun_preceders = [a[1] for (a, b) in word_tag_pairs if b[1] == 'JJ']
#print(noun_preceders)
fdist = nltk.FreqDist(noun_preceders)
[tag for (tag, _) in fdist.most_common()]

In [ ]:
# what are teh most common verbs
word_tag_fd = nltk.FreqDist(text_tagged)
[wt[0] for (wt, _) in word_tag_fd.most_common() if wt[1] == 'VB']

In [ ]:
# most likely words for a given tag VBN = 
cfd2 = nltk.ConditionalFreqDist((tag, word) for (word, tag) in text_tagged)
list(cfd2['VBN'])

In [ ]:
#find vbd (past tense) and VBN (past participle), find words which can be both and see surrounding text
nltk.help.upenn_tagset('VB.*')
cfd1 = nltk.ConditionalFreqDist(text_tagged)
[w for w in cfd1.conditions() if 'VBD' in cfd1[w] and 'VBN' in cfd1[w]]
idx1 = text_tagged.index(('localized', 'VBD'))
print(text_tagged[idx1-4:idx1+1])
idx2 = text_tagged.index(('localized', 'VBN'))
print(text_tagged[idx2-4:idx2+1])